In [1]:
# v1.0.7 - 14/02/2020
import numpy as np
import pandas as pd
pd.set_option("display.max_columns",1000)
pd.set_option("display.max_rows",1000)
import geopandas as gpd

from shapely.geometry import Point
crs = {"init":"epsg:4326"}

# ----------------------------------------------------------------------------------------------------------

import matplotlib.pyplot as plt
plt.style.use("ggplot") # to make matplotlib look better
%matplotlib inline
import seaborn as sns

import plotly
import chart_studio
import cufflinks as cf
cf.set_config_file(offline=True) # to make cufflinks work offline

import pandas_bokeh
pandas_bokeh.output_notebook()

#from plotly import colors.DEFAULT_PLOTLY_COLORS # to know default plotly colors
from plotly import graph_objs as go
BKCF_colors = ['#e32592','#fae700','#34b233','#58585a','#939597','#E0E1DD','#5bcbf5','#fe4540','#c7be71'] # to use with cufflinks
layout = go.Layout(colorway=BKCF_colors) # to use with plain plotly
from plotly import tools as tls
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True) # to allow plotly offline for jupyter notebooks
from bokeh.plotting import save # to allow saving pandas bokeh plots into html files
import plotly.express as px
import yfinance as yf
from datetime import date
import calendar
# ----------------------------------------------------------------------------------------------------------
import pickle
from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:95% !important; }</style>")) # to make jupyter notebook appearence bigger

Loading BokehJS ...

In [2]:
#Save in pickle format
def save_obj(obj, name, path="./obj"):
    if not(os.path.isdir(path)):
        os.mkdir(path)
        print("Creado directorio './obj' donde guardar objeto")
    with open("{}/{}.pkl".format(path,name), "wb") as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
#Load a pickle        
def load_obj(name, path="./obj"):
    with open("{}/{}.pkl".format(path, name), "rb") as f:
        return pickle.load(f)
    
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
df_metricas = load_obj("df_metricas_sp")

#Sustituimos por NaN los valores que son none, null y los empty
df_metricas = df_metricas.replace([None],"NaN").replace('null',"NaN").replace("","NaN")
lista_columnas_float = list(df_metricas.columns)
lista_columnas_float.remove('symbol')
lista_columnas_float.remove('ceo')
lista_columnas_float.remove('industry')
lista_columnas_float.remove('sector')
lista_columnas_float.remove('date')
df_metricas[lista_columnas_float] = df_metricas[lista_columnas_float].astype(float)
df_metricas['year'] = pd.DatetimeIndex(df_metricas['date']).year
df_metricas = df_metricas[df_metricas['symbol']!="NTRS"]
df_metricas.head()

,symbol,beta,ceo,industry,sector,volAvg,date,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,Income Tax Expense,Net Income - Non-Controlling int,Net Income - Discontinued ops,Net Income,Preferred Dividends,Net Income Com,EPS,EPS Diluted,Weighted Average Shs Out,Weighted Average Shs Out (Dil),Dividend per Share,Gross Margin,EBITDA Margin,EBIT Margin,Profit Margin,Free Cash Flow margin,EBITDA,EBIT,Consolidated Income,Earnings Before Tax Margin,Net Profit Margin,Cash and cash equivalents,Short-term investments,Cash and short-term investments,Receivables,Inventories,Total current assets,"Property, Plant & Equipment Net",Goodwill and Intangible Assets,Long-term investments,Tax assets,Total non-current assets,Total assets,Payables,Short-term debt,Total current liabilities,Long-term debt,Total debt,Deferred revenue,Tax Liabilities,Deposit Liabilities,Total non-current liabilities,Total liabilities,Other comprehensive income,Retained earnings (deficit),Total shareholders equity,Investments,Net Debt,Other Assets,Other Liabilities,Depreciation & Amortization,Stock-based compensation,Operating Cash Flow,Capital Expenditure,Acquisitions and disposals,Investment purchases and sales,Investing Cash flow,Issuance (repayment) of debt,Issuance (buybacks) of shares,Dividend payments,Financing Cash Flow,Effect of forex changes on cash,Net cash flow / Change in cash,Free Cash Flow,Net Cash/Marketcap,priceBookValueRatio,priceToBookRatio,priceToSalesRatio,priceEarningsRatio,receivablesTurnover_x,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue,ebitperRevenue,ebtperEBIT,niperEBT,grossProfitMargin,operatingProfitMargin,pretaxProfitMargin,netProfitMargin,effectiveTaxRate,returnOnAssets,returnOnEquity,returnOnCapitalEmployed,nIperEBT,eBTperEBIT,eBITperRevenue,receivablesTurnover_y,payablesTurnover,inventoryTurnover,fixedAssetTurnover,assetTurnover,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,daysOfPayablesOutstanding,cashConversionCycle,debtRatio,debtEquityRatio,longtermDebtToCapitalization,totalDebtToCapitalization,interestCoverage,cashFlowToDebtRatio,companyEquityMultiplier,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,payoutRatio,receivablesTurnover,operatingCashFlowSalesRatio,freeCashFlowOperatingCashFlowRatio,cashFlowCoverageRatios,shortTermCoverageRatios,capitalExpenditureCoverageRatios,dividendpaidAndCapexCoverageRatios,dividendPayoutRatio,Gross Profit Growth,EBIT Growth,Operating Income Growth,Net Income Growth,EPS Growth,EPS Diluted Growth,Weighted Average Shares Growth,Weighted Average Shares Diluted Growth,Dividends per Share Growth,Operating Cash Flow growth,Free Cash Flow growth,10Y Revenue Growth (per Share),5Y Revenue Growth (per Share),3Y Revenue Growth (per Share),10Y Operating CF Growth (per Share),5Y Operating CF Growth (per Share),3Y Operating CF Growth (per Share),10Y Net Income Growth (per Share),5Y Net Income Growth (per Share),3Y Net Income Growth (per Share),10Y Shareholders Equity Growth (per Share),5Y Shareholders Equity Growth (per Share),3Y Shareholders Equity Growth (per Share),10Y Dividend per Share Growth (per Share),5Y Dividend per Share Growth (per Share),3Y Dividend per Share Growth (per Share),Receivables growth,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,Revenue per Share,Net Income per Share,Operating Cash Flow per Share,Free Cash Flow per Share,Cash per Share,Book Value per Share,Tangible Book Value per Share,Shareholders Equity per Share,Interest Debt per Share,Market Cap,Enterprise Value,PE ratio,Price to Sales Ratio,POCF ratio,PFCF ratio,PB ratio,PTB ratio,EV to Sales,Enterprise Value over EBITDA,EV to Operating cash flow,EV to Free cash flow,Earnings Yield,F

In [4]:
#df_metricas[['symbol','sector']].drop_duplicates().groupby("sector").count().reset_index().sort_values(by="symbol",ascending=False)
#df_metricas[['symbol','industry']].drop_duplicates().groupby("industry").count().reset_index().sort_values(by="symbol",ascending=False)

# Metrics by sector

In [5]:
df_1 = pd.DataFrame(df_metricas.groupby(["year","sector"])['Market Cap','Revenue', 'Net Income'].sum()).reset_index()
df_1= df_1[(df_1['sector']!="NaN")&(df_1['year']<=2018)]

helper_1 = df_1.groupby('year')['Market Cap','Revenue','Net Income'].\
    agg('sum').reset_index().rename(columns={'Market Cap':'Sum_market_cap','Revenue':'Sum_revenue','Net Income':'Sum_net_income'})
helper_1.head(10)

df_1 =df_1.merge(helper_1)

df_1['Relative_market_cap'] = df_1['Market Cap']*100/df_1['Sum_market_cap']
df_1['Relative_revenue'] = df_1['Revenue']*100/df_1['Sum_revenue']
df_1['Relative_net_income'] = df_1['Net Income']*100/df_1['Sum_net_income']

df_1.head(20)

,year,sector,Market Cap,Revenue,Net Income,Sum_market_cap,Sum_revenue,Sum_net_income,Relative_market_cap,Relative_revenue,Relative_net_income
0,2009,Basic Materials,2.238894e+11,2.188823e+11,5.859802e+09,8.102190e+12,6.300777e+12,4.233839e+11,2.763320,3.473894,1.384040
1,2009,Communication Services,2.795924e+11,2.509090e+11,1.853613e+10,8.102190e+12,6.300777e+12,4.233839e+11,3.450825,3.982192,4.378090
2,2009,Consumer Cyclical,5.168301e+11,4.982796e+11,2.341369e+10,8.102190e+12,6.300777e+12,4.233839e+11,6.378894,7.908224,5.530134
3,2009,Consumer Defensive,7.713372e+11,5.881110e+11,5.252357e+10,8.102190e+12,6.300777e+12,4.233839e+11,9.520108,9.333943,12.405662
4,2009,Energy,1.033407e+12,1.026162e+12,4.281917e+10,8.102190e+12,6.300777e+12,4.233839e+11,12.754662,16.286280,10.113558
5,2009,Financial Services,1.389071e+12,9.545676e+11,5.287795e+10,8.102190e+12,6.300777e+12,4.233839e+11,17.144389,15.149997,12.489362
6,2009,Healthcare,9.203762e+11,8.559956e+11,7.830622e+10,8.102190e+12,6.300777e+12,4.233839e+11,11.359598,13.585556,18.495325
7,2009,Industrials,9.850110e+11,9.764624e+11,4.906216e+10,8.102190e+12,6.300777e+12,4.233839e+11,12.157343,15.497492,11.588103
8,2009,Real Estate,1.499598e+11,3.438135e+10,2.567748e+09,8.102190e+12,6.300777e+12,4.233839e+11,1.850855,0.545668,0.606482
9,2009,Technology,1.516484e+12,6.307988e+11,7.449512e+10,8.102190e+12,6.300777e+12,4.233839e+11,18.716964,10.011444,17.595170


In [9]:
df_1

,year,sector,Market Cap,Revenue,Net Income,Sum_market_cap,Sum_revenue,Sum_net_income,Relative_market_cap,Relative_revenue,Relative_net_income
0,2009,Basic Materials,2.238894e+11,2.188823e+11,5.859802e+09,8.102190e+12,6.300777e+12,4.233839e+11,2.763320,3.473894,1.384040
1,2009,Communication Services,2.795924e+11,2.509090e+11,1.853613e+10,8.102190e+12,6.300777e+12,4.233839e+11,3.450825,3.982192,4.378090
2,2009,Consumer Cyclical,5.168301e+11,4.982796e+11,2.341369e+10,8.102190e+12,6.300777e+12,4.233839e+11,6.378894,7.908224,5.530134
3,2009,Consumer Defensive,7.713372e+11,5.881110e+11,5.252357e+10,8.102190e+12,6.300777e+12,4.233839e+11,9.520108,9.333943,12.405662
4,2009,Energy,1.033407e+12,1.026162e+12,4.281917e+10,8.102190e+12,6.300777e+12,4.233839e+11,12.754662,16.286280,10.113558
5,2009,Financial Services,1.389071e+12,9.545676e+11,5.287795e+10,8.102190e+12,6.300777e+12,4.233839e+11,17.144389,15.149997,12.489362
6,2009,Healthcare,9.203762e+11,8.559956e+11,7.830622e+10,8.102190e+12,6.300777e+12,4.233839e+11,11.359598,13.585556,18.495325
7,2009,Industrials,9.850110e+11,9.764624e+11,4.906216e+10,8.102190e+12,6.300777e+12,4.233839e+11,12.157343,15.497492,11.588103
8,2009,Real Estate,1.499598e+11,3.438135e+10,2.567748e+09,8.102190e+12,6.300777e+12,4.233839e+11,1.850855,0.545668,0.606482
9,2009,Technology,1.516484e+12,6.307988e+11,7.449512e+10,8.102190e+12,6.300777e+12,4.233839e+11,18.716964,10.011444,17.595170


## Market cap

In [14]:
#Stacked barplot absolute
stacked_bar_absolute = px.area(df_1, x="year", y="Market Cap", color="sector", title="Evolution of market cap")

#Stacked barplot relative
stacked_bar_relative = px.bar(df_1, x="year", y="Relative_market_cap", color="sector", title="Evolution of market cap")

#Grouped barplot
grouped_bar_absolute = px.bar(df_1, x="year", y="Market Cap", color="sector",barmode="group",title="Evolution of market cap")


stacked_bar_absolute.show()
stacked_bar_relative.show()
grouped_bar_absolute.show()

## Revenue

In [7]:
#Stacked barplot absolute
stacked_bar_absolute = px.bar(df_1, x="year", y="Revenue", color="sector", title="Evolution of revenue")

#Stacked barplot relative
stacked_bar_relative = px.bar(df_1, x="year", y="Relative_revenue", color="sector", title="Evolution of revenue")

#Grouped barplot
grouped_bar_absolute = px.bar(df_1, x="year", y="Revenue", color="sector",barmode="group",title="Evolution of revenue")


stacked_bar_absolute.show()
stacked_bar_relative.show()
grouped_bar_absolute.show()

## Net income

In [8]:
#Stacked barplot absolute
stacked_bar_absolute = px.bar(df_1, x="year", y="Net Income", color="sector", title="Evolution of net income")

#Stacked barplot relative
stacked_bar_relative = px.bar(df_1, x="year", y="Relative_net_income", color="sector", title="Evolution of net income")

#Grouped barplot
grouped_bar_absolute = px.bar(df_1, x="year", y="Net Income", color="sector",barmode="group",title="Evolution of net income")


stacked_bar_absolute.show()
stacked_bar_relative.show()
grouped_bar_absolute.show()

## Gross margin

## EBITDA margin

## Net profit margin